In [11]:
from qiskit import Aer, execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info import Statevector
import numpy as np
from math import pi, sqrt, exp

In [12]:
def qft_inverse(qc, n):
    # Use swap only when qpe is setting  least significant qubit q0 with CU^(2^0) and nth qubit with CU^(2^(n-1))
    # as the iqft expect the information coded in phase basis in reverse order 
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    # perform inverse QFT to extract the information coded in phase basis and to measure it in computational basis  
    for j in range(n):
        for m in range(j):
            qc.cp(-math.pi/float(2**(j-m)), m, j)
        qc.h(j)

In [13]:
# This function will take the phase factor theta (value between 0 to 1) as input and will return a control gate 
# (unitary operator) that, when applied to the eignevector (|1> in this case) will kickback an eigenvalue of
# e^2pi*phs_factor to control qubit,  when the  eigenvalue is unknown, it can be found with the phase estimation 
def phase_gate(phs_factor):
    phase = phs_factor*(2*pi)
    qc1 = QuantumCircuit(2)
    qc1.cp(phase,0,1)
    CU_q = qc1.to_gate()
    return CU_q

In [14]:
def qpe_qiskit(control, target, circuit,CU):

    N = control.size
    
    #Apply Hadamard to all control qubits
    circuit.h(control)
   
    #Apply CU gates with various controls qubits 
    for i in range(N):
        CUi = CU.power(2**i)
        # swap will be needed in inverse qft circuit as this line of code in qpe circuit will set least significant 
        # qubit with CU^2 and most significant with CU^(2^(N-1)   
        circuit.append(CUi, [i,target]) # swap will be needed in inverse qft circuit as this line of code in qpe will set  
        # Comment the swap section in QFT inverse if you prefer the to do qpe in reveesed order, the line of code below will
        # set the most significant qubit with CU^2 and the least significant ubit with CU^(2^(N-1)
        # circuit.append(CUi, [control[control.size-i-1],target])

In [30]:
# Run qpe for on a single qubit target  
t=8 #Number of qubits in the control register
n=1 #Number of qubits in the register storing eigenvector

theta_factor = 1/10 # phase angle of 45 degree
CU = phase_gate(theta_factor)   # get the unitary

qr_control = QuantumRegister(t,'control')
qr_target = QuantumRegister(n,'target')
cr_control = ClassicalRegister(t,'meas_control')

qcr = QuantumCircuit(qr_control, qr_target, cr_control)
qcr.x(qr_target)   #Set target qubit to state |1> or the eignevector

#Apply single target QPE
qpe_qiskit(qr_control, qr_target, qcr,CU)

qcr.barrier(); qft_inverse(qcr, t); qcr.barrier()

# take measurement
qcr.measure(range(t),range(t))
# runt the circuit shots times
bkend = Aer.get_backend('qasm_simulator')
# collect and print the results
cnts =  execute(qcr, bkend,shots=100).result().get_counts()
print(cnts)

{'00010101': 1, '00010110': 1, '00010111': 1, '00011000': 2, '00011001': 19, '00011010': 68, '00011011': 4, '00011100': 1, '00011101': 1, '10011100': 1, '11000101': 1}


In [31]:
#what we have measured is the portion of a circle and the factor is the portion/total_values = val/2^N 
# translate the value of phase in decimal 
max_val = max(cnts, key=cnts.get)
max_val_dec = int(max_val, 2)
rad = max_val_dec * (1/2**t)*2*pi
print(' decimal value of output is', max_val_dec)
given_rad = theta_factor*2*pi
print('Given phase of unitray is ', given_rad , ' radians and ', math.degrees(given_rad), 'degrees')
print('estimated phase of unitray is ', rad , ' radians and ', math.degrees(rad), 'degrees')

 decimal value of output is 26
Given phase of unitray is  0.6283185307179586  radians and  36.0 degrees
estimated phase of unitray is  0.6381360077604268  radians and  36.5625 degrees


In [ ]:
# Run QPE for a single qubit target register
t=6 #Number of qubits in the control register
n=2 #Number of qubits in the register storing eigenvector

theta = 1/8 # phase angle of 45 degree
C_Ux = phase_gate_multitarget(theta, n)
#C_Ux = mqpcg(theta,n) # without using control for target = 2
qr_control = QuantumRegister(t,'control')
qr_target = QuantumRegister(n,'target')
cr_control = ClassicalRegister(t,'meas_control')

qcr = QuantumCircuit(qr_control, qr_target, cr_control)
#Set target qubit to state |11> 
qcr.x(qr_target)

#Apply QPE 
qpe_qiskit_multi(t,qr_control, qr_target, qcr, C_Ux)

qcr.barrier()
# Apply inverse QFT
qft_inverse(qcr, t)
# Measure
qcr.barrier()
qcr.measure(range(t),range(t))
bkend = Aer.get_backend('qasm_simulator')
cnts =  execute(qcr,bkend,shots=100).result().get_counts()
qcr.draw()

In [39]:
def phase_gate_multitarget(phs_factor, target_cnt):

    phase = phs_factor*(2*pi)

    # Earlier used cirtcuit to create gates but it is not creating unitary properly hence used operator
    #    qc1 = QuantumCircuit(target_cnt)  
    k = 2**target_cnt
    u = np.zeros([k, k], dtype = complex) 

    # set the diagonal elements to ones
    for i in range(k):
        u[i][i] = 1

    # set the last element as e^2pi*theta (phase)
    u[k-1][k-1] = np.exp(1.j*phase)
    
    m_CU = Operator(u)
    m_gate = UnitaryGate(m_CU, 'CU_g')
    
    # add control to make it multitarget control unitary that can operate on state |psi>
    mc_gate = m_gate.control() 

    return mc_gate

In [40]:
# define qpe for multiqubit target register 
def qpe_qiskit_multi(control, target, circuit, CU):

    N = control.size
    #Apply Hadamard to all control qubits
    circuit.h(control)
   
    #Apply CU gates with various controls qubits 
    for i in range(N):
        CUi = CU.power(2**i)
        ar=[]
        ar.append(i)
        ar.extend(range(N, N+target.size))
    # enable swap if you use the below line of code to correct the order of inverse QFT
        circuit.append(CUi, ar)

    # uncomment below line if qn (most significant bit) is set to CU^1 (least significant value of 
    # phase factor encoded in fourier basis) in this case a swap is not required in iqft    
    # circuit.append(CUi, [control.size-i-1],ar) # adjust ar to set the control bit

In [48]:
# Rune qpe for  multiqubit target the unitary will dd phase on on statevector |11...>

t=10 #Number of qubits in the control register
n=2 #Number of qubits in the register storing eigenvector
theta_factor = 1/50 # phase angle of 45 degree
C_Ux = phase_gate_multitarget(theta_factor, n)

#C_Ux = mqpcg(theta,n) # without using control for target = 2
qr_control = QuantumRegister(t,'control')
qr_target = QuantumRegister(n,'target')
cr_control = ClassicalRegister(t,'meas_control')

qcr = QuantumCircuit(qr_control, qr_target, cr_control)

#Set target qubit to eigenstate |11..n> 
qcr.x(qr_target)

#Apply QPE 
qpe_qiskit_multi(qr_control, qr_target, qcr, C_Ux)

qcr.barrier()
# call inverse qft 
qft_inverse(qcr, t)
qcr.barrier()

# take measurement 
qcr.measure(range(t),range(t))
bkend = Aer.get_backend('qasm_simulator')
cnts =  execute(qcr,bkend,shots=100).result().get_counts()
print(cnts)


{'0000000000': 1, '0000010001': 1, '0000010010': 4, '0000010011': 3, '0000010100': 48, '0000010101': 31, '0000010110': 3, '0000010111': 3, '0000011010': 1, '0000011011': 1, '0000011100': 1, '0000100010': 1, '0000100111': 1, '0000111110': 1}


In [49]:
#what we have measured is the portion of a circle and the factor is the portion/total_values = val/2^N 
# translate the value of phase in decimal 
max_val = max(cnts, key=cnts.get)
max_val_dec = int(max_val, 2)
rad = max_val_dec * (1/2**t)*2*pi
print(' decimal value of output is', max_val_dec)
given_rad = theta_factor*2*pi
print('Given phase of unitray is ', given_rad , ' radians and ', math.degrees(given_rad), 'degrees')
print('estimated phase of unitray is ', rad , ' radians and ', math.degrees(rad), 'degrees')

 decimal value of output is 20
Given phase of unitray is  0.12566370614359174  radians and  7.2 degrees
estimated phase of unitray is  0.1227184630308513  radians and  7.03125 degrees
